In [1]:
import json
from pathlib import Path

from cdp_backend.pipeline.transcript_model import SectionAnnotation, Transcript, TranscriptAnnotations

###############################################################################
# Parameters

# Set ANNOTATION_TARGET to the transcript / metadata prefix you want to annotate
ANNOTATION_TARGET = "01df39c18aea"

###############################################################################
# Globals

UNANNOTATED_DATASET = Path("unannotated").resolve(strict=True)
ANNOTATED_DATASET = Path("annotated").resolve()
ANNOTATED_DATASET.mkdir(exist_ok=True)

TARGET_METADATA_PATH = UNANNOTATED_DATASET / f"{ANNOTATION_TARGET}-metadata.json"
with open(TARGET_METADATA_PATH, "r") as open_target_metadata_file:
    TARGET_METADATA = json.load(open_target_metadata_file)
    
UNANNOTATED_TRANSCRIPT_PATH = UNANNOTATED_DATASET / TARGET_METADATA["transcript_name"]
with open(UNANNOTATED_TRANSCRIPT_PATH, "r") as open_unannotated_transcript_file:
    UNANNOTATED_TRANSCRIPT = Transcript.from_json(open_unannotated_transcript_file.read())

In [2]:
from typing import List, NamedTuple, Optional

class SectionTarget(NamedTuple):
    index: int
    name: str
    description: Optional[str]

section_targets: List[SectionTarget] = []
for i, emi in enumerate(sorted(TARGET_METADATA["event_minutes_items"], key=lambda emi: emi["index"])):
    section_targets.append(
        SectionTarget(
            index=i,
            name=emi["minutes_item_ref"]["name"],
            description=emi["minutes_item_ref"]["description"]
        )
    )
        
section_targets = sorted(section_targets, key=lambda t: t.index)
section_targets

[SectionTarget(index=0, name='Call To Order', description=None),
 SectionTarget(index=1, name='Approval of the Agenda', description=None),
 SectionTarget(index=2, name="Chair's Report", description=None),
 SectionTarget(index=3, name='Public Comment', description=None),
 SectionTarget(index=4, name='Inf 1843', description='Seattle Public Library 2020 Levy Report'),
 SectionTarget(index=5, name='Inf 1844', description='Seattle Public Library 2020 RSJI Report'),
 SectionTarget(index=6, name='Inf 1845', description='Seattle Center 2020 RSJI Report'),
 SectionTarget(index=7, name='Inf 1846', description='Seattle Parks and Recreation 2020 RSJI Report'),
 SectionTarget(index=8, name='Adjournment', description=None)]

In [3]:
# Add your annotations here

UNANNOTATED_TRANSCRIPT.annotations = {}
UNANNOTATED_TRANSCRIPT.annotations = TranscriptAnnotations(sections=[
    SectionAnnotation(
        name="Call to Order",
        start_sentence_index=0,
        stop_sentence_index=8,
        generator="Jackson Maxfield Brown"
    ),
    SectionAnnotation(
        name="Approval of the Agenda",
        start_sentence_index=8,
        stop_sentence_index=10,
        generator="Jackson Maxfield Brown",
    ),
    SectionAnnotation(
        name="Chair's Report",
        start_sentence_index=10,
        stop_sentence_index=22,
        generator="Jackson Maxfield Brown",
    ),
    SectionAnnotation(
        name="Public Comment",
        start_sentence_index=22,
        stop_sentence_index=60,
        generator="Jackson Maxfield Brown",
    ),
    SectionAnnotation(
        name="Inf 1843",
        start_sentence_index=60,
        stop_sentence_index=168,
        generator="Jackson Maxfield Brown",
        description="Seattle Public Library 2020 Levy Report",
    ),
    SectionAnnotation(
        name="Inf 1844",
        start_sentence_index=168,
        stop_sentence_index=325,
        generator="Jackson Maxfield Brown",
        description="Seattle Public Library 2020 RSJI Report",
    ),
    SectionAnnotation(
        name="Inf 1845",
        start_sentence_index=325,
        stop_sentence_index=436,
        generator="Jackson Maxfield Brown",
        description="Seattle Center 2020 RSJI Report",
    ),
    SectionAnnotation(
        name="Inf 1846",
        start_sentence_index=436,
        stop_sentence_index=584,
        generator="Jackson Maxfield Brown",
        description="Seattle Parks and Recreation 2020 RSJI Report",
    ),
    SectionAnnotation(
        name="Adjournment",
        start_sentence_index=584,
        stop_sentence_index=len(UNANNOTATED_TRANSCRIPT.sentences),
        generator="Jackson Maxfield Brown",
    ),
])

In [4]:
# Sanity check sections
for section_anno in UNANNOTATED_TRANSCRIPT.annotations.sections:
    print(section_anno.name)
    section = UNANNOTATED_TRANSCRIPT.sentences[section_anno.start_sentence_index:section_anno.stop_sentence_index]
    print("start --", section[0].text[:100])
    print("end   --", section[-1].text[:100])
    print("-" * 80)

Call to Order
start -- I am Council member Juarez, chair of the committee, will the clerk please call the role.
end   -- Council member Mosqueda is excused.
--------------------------------------------------------------------------------
Approval of the Agenda
start -- Hearing no objection.
end   -- The agenda is adopted.
--------------------------------------------------------------------------------
Chair's Report
start -- Let's move on to the chair's report.
end   -- At this time we will open the remote public comment period.
--------------------------------------------------------------------------------
Public Comment
start -- It remains the strong intent of the Seattle city council have public comment regularly included on m
end   -- Moving on to items of business.
--------------------------------------------------------------------------------
Inf 1843
start -- Nagine can you read item one in the record.
end   -- Any questions for my colleagues? seeing no questions, is there any

In [5]:
# Store to annotated dataset dir
ANNOTATED_TRANSCRIPT_PATH = ANNOTATED_DATASET / TARGET_METADATA["transcript_name"]
with open(ANNOTATED_TRANSCRIPT_PATH, "w") as open_annotated_transcript_file:
    open_annotated_transcript_file.write(UNANNOTATED_TRANSCRIPT.to_json())